In [1]:
# Define paths to the training, validation, and test datasets
train_folder='./dataset/train'
test_folder='./dataset/test'
validate_folder = './dataset/valid'

In [1]:
# Define paths to the specific classes within the dataset
normal_folder = './datset/valid/normal'
adenocarcinoma_folder = './datset/valid/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib'
large_cell_carcinoma_folder = './datset/valid/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa'
squamous_cell_carcinoma_folder = './datset/valid/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa'

In [14]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout2D, Activation, Lambda, Flatten, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import utils


ModuleNotFoundError: No module named 'distutils'

In [3]:
print("Libraries Imported")

Libraries Imported
